### 1. 데이터 불러오기 & 정리

In [ ]:
# prompt: 구글 드라이브 연결

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install -q -U tensorflow-text
!pip install -q tf-models-official

In [ ]:
!pip install -q transformers

In [ ]:
# 라이브러리 import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
import seaborn as sns

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/특정 테스크에 해당하는 자연어처리 모델 서빙하기/mrus_data.csv')
#data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/특정 테스크에 해당하는 자연어처리 모델 서빙하기/data.csv')
# data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/특정 테스크에 해당하는 자연어처리 모델 서빙하기/data_resampled.csv')

In [ ]:
from sklearn.model_selection import train_test_split

# 학습 데이터와 테스트 데이터로 분할 (여기서는 비율을 90:10으로 설정)
train_data, test_data = train_test_split(data, test_size=0.1, random_state=42)

In [ ]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

# 라벨을 정수로 매핑
label_encoder = LabelEncoder()
train_encoded = label_encoder.fit_transform(train_data['labels'])
test_encoded = label_encoder.transform(test_data['labels'])

# 원-핫 인코딩 수행
train_onehot_encoded = to_categorical(train_encoded)
test_onehot_encoded = to_categorical(test_encoded)

print(train_onehot_encoded)
print(test_onehot_encoded)

[[0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 ...
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]]
[[0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 ...
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]]


### 2. klue 모델 학습

In [ ]:
from transformers import AutoModel, AutoTokenizer
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

# klue 모델 불러오기
model = BertForSequenceClassification.from_pretrained("klue/bert-base", num_labels=3)
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

AUTOTUNE = tf.data.AUTOTUNE   #자동으로 GPU 성능 최적화

class SentimentDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float)
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
# tokenizer 함수에 리스트 전달
train_inputs = tokenizer(train_data['text'].tolist(), return_tensors='pt', padding=True, truncation=True, max_length=150)
test_inputs = tokenizer(test_data['text'].tolist(), return_tensors='pt', padding=True, truncation=True, max_length=150)

In [ ]:
train_dataset = SentimentDataset(train_inputs, train_onehot_encoded)
test_dataset = SentimentDataset(test_inputs, test_onehot_encoded)

In [ ]:
!pip install transformers[torch] -U

In [ ]:
len(train_data), len(test_data)

(131249, 14584)

In [ ]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # 모델과 학습 로그가 저장될 경로
    num_train_epochs=5,              # 학습 에폭 수
    per_device_train_batch_size=16,   # 훈련 배치 크기
    per_device_eval_batch_size=16,    # 평가 배치 크기
    warmup_steps=500,                # 웜업 스텝 수
    weight_decay=0.01,               # 가중치 감쇠
    logging_dir='./logs',            # 로그가 저장될 경로
    logging_steps=100,                # 로그 기록 간격
)

In [ ]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments
import torch

# 디바이스 설정
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

# Trainer 객체 초기화
trainer = Trainer(
    model=model,                         # 사용할 모델
    args=training_args,                  # 훈련 및 평가 설정
    train_dataset=train_dataset,        # 훈련 데이터셋
    eval_dataset=test_dataset           # 검증 데이터셋
)

# 모델 훈련
trainer.train()

<ipython-input-17-185134e25dbb>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Step,Training Loss
100,0.597400
200,0.364100
300,0.327200
400,0.314700
500,0.317600
600,0.310800
700,0.302900
800,0.308400
900,0.282900
1000,0.283300


<ipython-input-17-185134e25dbb>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-17-185134e25dbb>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-17-185134e25dbb>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-17-185134e25dbb>:12: UserWarning: To copy construct fro

RuntimeError: [enforce fail at inline_container.cc:424] . unexpected pos 819023360 vs 819023248

### 3. 예측 수행

In [ ]:
# 모델 저장 디렉토리 지정
save_directory = "/content/drive/MyDrive/Colab Notebooks/특정 테스크에 해당하는 자연어처리 모델 서빙하기/모델 관련/first_klue"

# 모델 저장
trainer.save_model(save_directory)

In [ ]:
trainer.evaluate(test_dataset)

<ipython-input-9-185134e25dbb>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


{'eval_loss': 0.44050419330596924,
 'eval_runtime': 31.1281,
 'eval_samples_per_second': 468.516,
 'eval_steps_per_second': 29.298,
 'epoch': 5.0}

In [ ]:
prediction = trainer.predict(test_dataset)

<ipython-input-17-185134e25dbb>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Step,Training Loss
100,0.597400
200,0.364100
300,0.327200
400,0.314700
500,0.317600
600,0.310800
700,0.302900
800,0.308400
900,0.282900
1000,0.283300


In [ ]:
trainer.model.classifier

Linear(in_features=768, out_features=3, bias=True)

In [ ]:
y_logit = torch.tensor(prediction[0])
y_logit[:10]

tensor([[ 5.1711, -5.9880, -6.2237],
        [ 6.0314, -6.2463, -7.6221],
        [-7.0070,  6.4722, -7.6987],
        [-7.7330, -6.7276,  6.7355],
        [-7.7703, -6.7119,  6.6977],
        [-7.6509, -6.8540,  6.7856],
        [-1.6979, -7.0933,  1.4933],
        [ 5.9789, -6.1761, -7.6901],
        [-5.3480,  5.4959, -8.8461],
        [-7.2975, -7.2949,  6.8887]])

In [ ]:
import torch.nn.functional as F

# 예측 데이터와 실제 데이터 비교
y_pred = F.softmax(y_logit, dim=-1).argmax(axis=1).numpy()
print(list(y_pred[:30]))
print(list(test_encoded[:30]))

[0, 0, 1, 2, 2, 2, 2, 0, 1, 2, 1, 0, 0, 0, 0, 2, 2, 1, 0, 0, 2, 0, 2, 2, 0, 2, 0, 2, 0, 1]
[1, 0, 2, 2, 2, 2, 2, 0, 0, 2, 1, 0, 0, 0, 0, 2, 2, 0, 0, 0, 2, 1, 0, 2, 0, 2, 0, 2, 0, 1]


In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score

# 정확도 출력
print(accuracy_score(test_encoded, y_pred))

cm=confusion_matrix(test_encoded, y_pred)
cm

0.8507953922106418


array([[4114,  538,  226],
       [ 468, 3885,  441],
       [ 141,  362, 4409]])